In [ ]:
import re
import pandas as pd
from tqdm import tqdm
from nltk import pos_tag
from pathlib import Path
from nltk.corpus import wordnet
from nltk.stem import SnowballStemmer, WordNetLemmatizer

In [2]:
# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger_eng')
# nltk.download('wordnet')
# nltk.download('omw-1.4')

In [ ]:
def tokenize(text):
    patterns = [
        (r'\b[a-zA-Z]+@[a-zA-Z]+\.[a-zA-Z]+\b', 'EMAIL'),
        (r'(?:\+7[-.\s]?)?\(?9\d{2}\)?[-.\s]?\d{3}[-.\s]?\d{2}[-.\s]?\d{2}|(?:8[-.\s]?)?\(?9\d{2}\)?\d{7}', 'PHONE'),
        (r'\b(?:Dr|Mr|Mrs|Ms|Prof)\.\s+[A-Z][a-zA-Z]*', 'ABBREVIATION'),
        (r'\b(?:Dr|Mr|Mrs|Ms|Prof|St)\.\s+[A-Z][a-zA-Z]*(?:\'[a-z]+)?', 'APPEAL_WITH_WORD'),
        (r'\$[\d,]+(?:\.\d{2})?|\d+\.\d{2}\$', 'CURRENCY'),
        (r'https?://[^\s<>"{}|\\^`\[\]]+', 'URL'),
        (r'^[a-zA-Z_]\w*\s*=\s*[\w\s()+\-*/^]+(?:\s*=\s*[\w\s()+\-*/^]+)*$', 'MATH'),
        (r'г\.\s+\w+[а-яё]*,\s+ул\.\s+\w+[а-яё]*(?:-\w+[а-яё]*)*,\s+д\.\s+\d+(?:,\s+кв\.\s+\d+)?', 'ADDRESS'),
        (r'[;:][-o]?[D\)\(PpFf3\[\]}{@\|\\\/]', 'EMOTICON')
    ]

    placeholders = {}
    counter = 0

    # заменяем совпадения на плейсхолдеры
    for pattern, tag in patterns:
        matches = re.findall(pattern, text, flags=re.IGNORECASE)
        for match in set(matches):
            ph = f"__{tag}_{counter}__"
            placeholders[ph] = match
            text = re.sub(re.escape(match), ph, text, count=1)
            counter += 1

    # сплит предложений
    parts = re.split(r'([.!?]+)', text)
    if len(parts) < 2:
        sentences = [text]
    else:
        sentences = ["".join(x) for x in zip(parts[0::2], parts[1::2])]
        if len(parts) % 2 != 0:
            sentences.append(parts[-1])

    # токенизация
    all_sentences = []
    for sent in sentences:
        tokens = re.findall(r"\w+(?:'\w+)?|[^\w\s]", sent)
        out = []
        for t in tokens:
            if t in placeholders:
                out.append(placeholders[t])
            else:
                out.append(t)
        all_sentences.append(out)

    return all_sentences

In [4]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
def process_tokens(sentences_list):
    stemmer = SnowballStemmer("english")
    lemmatizer = WordNetLemmatizer()

    final = []

    for sent in sentences_list:
        # определяем части речи для каждого слова в предложении
        tagged = pos_tag(sent)
        row = []

        # для каждого слова и его части речи
        for word, pos in tagged:
            # если слово не состоит из букв, оставляем как есть
            if not word.isalpha():
                stem = word
                lemma = word
            else:
                # вычисляем основу слова двумя способами
                stem = stemmer.stem(word.lower())
                lemma = lemmatizer.lemmatize(word.lower(), get_wordnet_pos(pos))

            row.append((word, stem, lemma))

        final.append(row)

    return final

In [ ]:
def sanitize_label(label):
    # заменяем символы на подчеркивание
    label = re.sub(r'[<>:"/\\|?*$]', '_', label)
    # убираем пробелы и точки в начале и конце
    label = label.strip().strip('.')
    # заменяем множественные подчеркивания и пробелы на одно подчеркивание
    label = re.sub(r'[_\s]+', '_', label)
    return label

In [ ]:
def save_annotated(documents):
    base = Path("./assets/annotated-corpus")

    for doc in documents:
        # получаем идентификатор документа и очищаем его
        doc_id = sanitize_label(str(doc["doc_id"]))
        label = str(doc["label"])
        split = doc["split"]
        # получаем обработанные предложения
        sentences = doc["sentences"]

        path = base / split / label
        path.mkdir(parents=True, exist_ok=True)

        with open(path / f"{doc_id}.tsv", "w", encoding="utf-8") as f:
            for i, sent in enumerate(sentences):
                if i > 0:
                    f.write("\n\n")
                for token, stem, lemma in sent:
                    f.write(f"{token}\t{stem}\t{lemma}\n")

In [ ]:
def process_dataset(name):
    docs = []
    df = pd.read_csv(f"../assets/{name}.csv", header=None, names=['class', 'title', 'text'])

    for i, row in tqdm(df.iterrows(), total=len(df)):
        text = row['text']

        # разбиваем текст на предложения и токены
        sentences = tokenize(text)
        # лемматизация, стемминг
        anno = process_tokens(sentences)

        docs.append({
            'doc_id': i,
            'label': row['class'],
            'split': name,
            'sentences': anno
        })

    save_annotated(docs)

In [9]:
process_dataset('train')

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120001/120001 [03:23<00:00, 591.00it/s]


In [10]:
process_dataset('test')

  1%|█▏                                                                                                                                                                                         | 49/7601 [00:00<00:15, 485.84it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7601/7601 [00:12<00:00, 597.35it/s]
